In [34]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import speech_recognition as sr
import os
import language_tool_python
import textstat


def video_to_audio(video_path, audio_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_path)
    audio_clip.close()

def recognize_speech(audio_path):
    recognizer = sr.Recognizer()
    
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Sorry, could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def split_audio(audio_path, segment_duration=10):
    audio = AudioSegment.from_wav(audio_path)
    segments = [audio[i:i+segment_duration*1000] for i in range(0, len(audio), segment_duration*1000)]
    return segments

if __name__ == "__main__":
    # Step 1: Take video file path as input
    video_path = "bad.mp4"

    # Step 2: Specify output audio file path
    output_directory = "Downloads"
    os.makedirs(output_directory, exist_ok=True)  # Create output directory if it doesn't exist
    audio_path = os.path.join(output_directory, "audio.wav")  # Output audio file path

    # Step 3: Convert video to audio
    video_to_audio(video_path, audio_path)
    print("Video converted to audio successfully.")

    # Step 4: Split audio into segments
    #  audio_segments = split_audio(audio_path)

    # Step 5: Recognize speech from each segment
    #      for i, segment in enumerate(audio_segments):
    #         segment_path = os.path.join(output_directory, f"segment_{i}.wav")
    #         segment.export(segment_path, format="wav")
    recognized_text = recognize_speech("Downloads/audio.wav")


##intonation checker 
import librosa
import numpy as np

audio_file = "Downloads/audio.wav"

y, sr = librosa.load(audio_file, sr=None)

pitch_values, _ = librosa.piptrack(y=y, sr=sr)

pitch_values_mean = pitch_values.mean(axis=0)

threshold_low_variation = 20  # Hz
threshold_high_variation = 40  # Hz
min_duration_low_variation = 3  # seconds
min_duration_high_variation = 0.2  # seconds

low_pitch_count = 0
high_pitch_count = 0

in_low_variation = False
in_high_variation = False

for pitch_value in pitch_values_mean:
    if pitch_value < threshold_low_variation:
        if not in_low_variation:
            in_low_variation = True
            low_pitch_count += 1
            in_high_variation = False
    elif pitch_value > threshold_high_variation:
        if not in_high_variation:
            in_high_variation = True
            high_pitch_count += 1
            in_low_variation = False
    else:
        in_low_variation = False
        in_high_variation = False
Intonation = ((high_pitch_count/low_pitch_count)*100)

#checking Grammer
def grammar_check(recognized_text):

    tool = language_tool_python.LanguageTool('en-US')

    matches = tool.check(recognized_text)

    for match in matches:
        print("Error:", match.ruleId)
        print("Message:", match.message)
        print("Suggestions:", match.replacements)
        print("-----------")

#checking Readability
def gf_score(recognized_text):
    gf_score=textstat.gunning_fog(recognized_text)
    # Interpret the GFI score and provide a meaningful output
    return gf_score

MoviePy - Writing audio in Downloads\audio.wav


MoviePy - Done.
Video converted to audio successfully.


In [35]:
print("Intonation Checker: ",Intonation)
print("Grammer checker: ",grammar_check(recognized_text))
print("Readability score :",gf_score(recognized_text))

Intonation Checker:  54.285714285714285
Error: UPPERCASE_SENTENCE_START
Message: This sentence does not start with an uppercase letter.
Suggestions: ['Good']
-----------
Grammer checker:  None
Readability chacker : 22.44
